## Mount GoogleDrive

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


## Parameters

In [ ]:
# Dataset
dataset_folder_path = '/content/gdrive/MyDrive/deephomography/dataset/'
rho = 32
patch_size = 128
image_size = (320,240)

# Create required folder(s)
!mkdir -p {dataset_folder_path}

## Download and extract COCO 2017 dataset

In [ ]:
!wget http://images.cocodataset.org/zips/train2017.zip
!wget http://images.cocodataset.org/zips/val2017.zip
!wget http://images.cocodataset.org/zips/test2017.zip

!echo "Extracting train2017.zip..."
!unzip -q train2017.zip

!echo "Extracting val2017.zip..."
!unzip -q val2017.zip

!echo "Extracting test2017.zip..."
!unzip -q test2017.zip

!echo "Deleting downloaded .zip files..."
!rm -f train2017.zip val2017.zip test2017.zip

--2022-07-10 17:53:21--  http://images.cocodataset.org/zips/train2017.zip
Resolving images.cocodataset.org (images.cocodataset.org)... 54.231.136.113
Connecting to images.cocodataset.org (images.cocodataset.org)|54.231.136.113|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 19336861798 (18G) [application/zip]
Saving to: ‘train2017.zip’

train2017.zip       100%[===================>]  18.01G  83.8MB/s    in 3m 48s  

2022-07-10 17:57:10 (80.7 MB/s) - ‘train2017.zip’ saved [19336861798/19336861798]

--2022-07-10 17:57:10--  http://images.cocodataset.org/zips/val2017.zip
Resolving images.cocodataset.org (images.cocodataset.org)... 54.231.204.185
Connecting to images.cocodataset.org (images.cocodataset.org)|54.231.204.185|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 815585330 (778M) [application/zip]
Saving to: ‘val2017.zip’

val2017.zip         100%[===================>] 777.80M  82.7MB/s    in 9.9s    

2022-07-10 17:57:20 (78.6 MB/s) 

## Data generation

In [ ]:
import os
import cv2
import random
import numpy as np
from tqdm import tqdm

# Do not change these limits! They prevent bordering artifacts
tl_point_limits = [(rho,rho), (image_size[0]-patch_size-rho,image_size[1]-patch_size-rho)]

# Function to generate a single sample
def generate_sample(image_path):
    # Read image file (as grayscale)
    image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)

    # Resize image to image_size
    image = cv2.resize(image, image_size, interpolation=cv2.INTER_CUBIC)

    # Select a random patch
    tl_point = np.random.randint(tl_point_limits[0], tl_point_limits[1], size=2)
    tr_point = tl_point + (patch_size,0)
    br_point = tl_point + (patch_size,patch_size)
    bl_point = tl_point + (0,patch_size)
    patch_points = np.array([tl_point, tr_point, br_point, bl_point]);

    # Extract patch
    patch = image[tl_point[1]:br_point[1], tl_point[0]:br_point[0]]

    # Create a random perturbation (or H_fp)
    perturbation = np.random.randint(-rho, rho+1, size=(4,2))
    perturbed_patch_points = patch_points + perturbation

    # Find corresponding homography and its inverse
    H = cv2.getPerspectiveTransform(patch_points.astype(np.float32), perturbed_patch_points.astype(np.float32))
    H_inv = np.linalg.inv(H)

    # Apply inverse homography to image
    warped_image = cv2.warpPerspective(image, H_inv, image_size)

    # Extract perturbed patch
    perturbed_patch = warped_image[tl_point[1]:br_point[1], tl_point[0]:br_point[0]]

    # Return generated sample
    return patch, perturbed_patch, patch_points, perturbation

def generate_samples_from_folder(folder_path, num_samples=0):
    # Create target folder
    target_folder_path = folder_path + 'homography'
    os.makedirs(target_folder_path, exist_ok=True)

    print('Generating ' + target_folder_path + '...')

    # List folder contents
    file_list = os.listdir(folder_path)
    if num_samples > len(file_list):
        file_list = random.choices(file_list, k=num_samples)
    elif num_samples > 0:
        file_list = random.sample(file_list, k=num_samples)

    # Process files 
    for count,file in enumerate(tqdm(file_list)):
        file_path = os.path.join(folder_path, file)
        target_file_path = os.path.join(target_folder_path, f'{count:08d}')

        # Save generated sample
        np.save(target_file_path, np.array((generate_sample(file_path)), dtype=object))

## Process folders and generate new data for homography estimation

In [ ]:
generate_samples_from_folder(folder_path='train2017')
generate_samples_from_folder(folder_path='val2017')
generate_samples_from_folder(folder_path='test2017')

!echo "Deleting dataset folders..."
!rm -dr train2017 val2017 test2017

Generating train2017homography...


100%|██████████| 118287/118287 [24:36<00:00, 80.14it/s]


Generating val2017homography...


100%|██████████| 5000/5000 [01:02<00:00, 80.25it/s] 


Generating test2017homography...


100%|██████████| 40670/40670 [07:55<00:00, 85.48it/s]


Deleting dataset folders...


## Tarball generated datasets

In [ ]:
!echo "Tarballing train2017homography..."
!tar -cf train2017homography.tar train2017homography

!echo "Tarballing val2017homography..."
!tar -cf val2017homography.tar val2017homography

!echo "Tarballing test2017homography..."
!tar -cf test2017homography.tar test2017homography

!echo "Deleting generated dataset folders..."
!rm -dr train2017homography val2017homography test2017homography

Tarballing train2017homography...
Tarballing val2017homography...
Tarballing test2017homography...
Deleting generated dataset folders...


## Move tarballs to GoogleDrive

In [ ]:
!echo "Moving train2017homography.tar..."
!mv train2017homography.tar {dataset_folder_path}

!echo "Moving val2017homography.tar..."
!mv val2017homography.tar {dataset_folder_path}

!echo "Moving test2017homography.tar..."
!mv test2017homography.tar {dataset_folder_path}

Moving train2017homography.tar...
Moving val2017homography.tar...
Moving test2017homography.tar...


## Unmount GoogleDrive

In [8]:
drive.flush_and_unmount()